In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"C:\Users\Admin\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from sqlalchemy import text, bindparam
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from moduly.ai import zaladuj_api_i_klienta, instrukcja_tlumacz, instrukcja_redaktor
from moduly.utils import sklej_warunki_w_WHERE, generuj_hash_djb2
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
from google import genai
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [3]:
silnik = utworz_engine_do_db()

In [ ]:
q_select_misje = text("""
    SELECT m.MISJA_ID_MOJE_PK
    FROM dbo.MISJE AS m
    WHERE 1=1
      AND m.DODATEK_EN = :dodatek_en
      AND m.MISJA_TYTUL_PL IS NOT NULL
""")

q_select_misje_tresc = text("""
    SELECT TECH_ID, TRESC
    FROM dbo.MISJE_STATUSY
    WHERE 1=1
      AND MISJA_ID_MOJE_FK IN :misje_id
      AND STATUS = '3_ZATWIERDZONO'
""").bindparams(bindparam("misje_id", expanding=True))

with silnik.connect() as conn:
    misje = conn.execute(q_select_misje, {"dodatek_en": "Midnight"}).scalars().all()
    misje_tresc = conn.execute(q_select_misje_tresc, {"misje_id": misje}).all()
    wynik = pd.DataFrame(misje_tresc).set_index("TECH_ID")
#print(wynik)

In [3]:
SCIEZKA = r"C:\Users\Admin\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\excel-zatwierdzenia\Midnight.xlsx"

In [7]:
keys = ["MISJA_ID", "SEGMENT", "PODSEGMENT", "ID_SEGMENTU", "NR_BLOKU", "NR_WYP"]

df = pd.read_excel(
    SCIEZKA,
    usecols=keys + ["STATUS", "TRESC"]
)

f = df[df["STATUS"].isin(["0_ORYGINAŁ", "2_ZREDAGOWANO"])].copy()

orig = f[f["STATUS"] == "0_ORYGINAŁ"].copy()
pl = f[f["STATUS"] == "2_ZREDAGOWANO"].copy()

orig["POS"] = orig.groupby(keys).cumcount()
pl["POS"] = pl.groupby(keys).cumcount()

pairs = orig.merge(
    pl,
    on=keys + ["POS"],
    how="inner",
    suffixes=("_ORIG", "_PL")
)

pairs = pairs[pairs["TRESC_PL"].notna()].copy()
pairs["HASH"] = pairs["TRESC_ORIG"].astype(str).str.lower().apply(generuj_hash_djb2)

slownik = dict(zip(pairs["HASH"], pairs["TRESC_PL"]))


In [8]:
with open("slownik.txt", "w", encoding="utf-8") as f:
    for h, tlumaczenie in slownik.items():
        if pd.isna(tlumaczenie):
            continue
        t = str(tlumaczenie).replace("\\", "\\\\").replace('"', '\\"')
        f.write(f'["{h}"] = "{t}",\n')
